In [1]:
!pip install numpy 
!pip install pandas
!pip install matplotlib 
!pip install seaborn 
!pip install sklearn
!pip install tensorflow


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier, StackingClassifier, GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from sklearn.svm import SVC

In [3]:
wine_ds = pd.read_csv('WQ_Dataset.csv')

In [4]:
x = wine_ds.drop("quality", axis=1)
y = wine_ds["quality"].apply(lambda y_value: "perfect" if y_value == 10 else ("good" if (y_value >= 7 and y_value < 10) else ("average" if (y_value >= 5 and y_value < 7) else ("bad" if (y_value >= 3 and y_value < 5) else "inedible"))))

In [5]:
y = pd.factorize(y)[0]
num_classes = len(np.unique(y))
y = to_categorical(y, num_classes = num_classes)

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=5)

In [7]:
rf = RandomForestClassifier(n_estimators=100, random_state=5)
rf.fit(x_train, np.argmax(y_train, axis=1))

RandomForestClassifier(random_state=5)

In [8]:
gbm = GradientBoostingClassifier(n_estimators=100, random_state=5)
gbm.fit(x_train, np.argmax(y_train, axis=1))

GradientBoostingClassifier(random_state=5)

In [9]:
svm = SVC(probability=True, random_state=5)
svm.fit(x_train, np.argmax(y_train, axis=1))

SVC(probability=True, random_state=5)

In [10]:
rf_train_preds = rf.predict_proba(x_train)
gbm_train_preds = gbm.predict_proba(x_train)
svm_train_preds = svm.predict_proba(x_train)

In [11]:
rf_test_preds = rf.predict_proba(x_test)
gbm_test_preds = gbm.predict_proba(x_test)
svm_test_preds = svm.predict_proba(x_test)

In [12]:
x_train_stack = np.hstack((rf_train_preds, gbm_train_preds, svm_train_preds))
x_test_stack = np.hstack((rf_test_preds, gbm_test_preds, svm_test_preds))

In [13]:
def create_meta_nn_model(input_dim):
    model = Sequential()
    model.add(Dense(32, input_dim=input_dim, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [14]:
meta_nn_model = create_meta_nn_model(x_train_stack.shape[1])
meta_nn_model.fit(x_train_stack, y_train, epochs=50, batch_size=10, verbose=1)



Epoch 1/50


128/128 [==============================] - 1s 3ms/step - loss: 0.6377 - accuracy: 0.7858
Epoch 2/50
128/128 [==============================] - 0s 2ms/step - loss: 0.1337 - accuracy: 0.9812
Epoch 3/50
128/128 [==============================] - 0s 1ms/step - loss: 0.0330 - accuracy: 0.9969
Epoch 4/50
128/128 [==============================] - 0s 1ms/step - loss: 0.0146 - accuracy: 0.9992
Epoch 5/50
128/128 [==============================] - 0s 2ms/step - loss: 0.0084 - accuracy: 1.0000
Epoch 6/50
128/128 [==============================] - 0s 2ms/step - loss: 0.0053 - accuracy: 1.0000
Epoch 7/50
128/128 [==============================] - 0s 2ms/step - loss: 0.0038 - accuracy: 1.0000
Epoch 8/50
128/128 [==============================] - 0s 2ms/step - loss: 0.0027 - accuracy: 1.0000
Epoch 9/50
128/128 [==============================] - 0s 2ms/step - loss: 0.0021 - accuracy: 1.0000
Epoch 10/50
128/128 [==============================] - 0s 2ms/step - loss: 0.0016 - accuracy: 1.

In [15]:
y_pred_stack = meta_nn_model.predict(x_test_stack)
y_pred_stack = np.argmax(y_pred_stack, axis=1)

10/10 [==============================] - 0s 2ms/step


In [16]:
print(f"Ensemble model accuracy: {accuracy_score(np.argmax(y_test, axis=1), y_pred_stack)}")
print(classification_report(np.argmax(y_test, axis=1), y_pred_stack))

Ensemble model accuracy: 0.884375
              precision    recall  f1-score   support

           0       0.90      0.97      0.93       267
           1       0.82      0.50      0.62        46
           2       0.25      0.14      0.18         7

    accuracy                           0.88       320
   macro avg       0.66      0.54      0.58       320
weighted avg       0.87      0.88      0.87       320

